In [1]:

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics
from distributions_comparison_utils import significantly_altered_reactions


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

In [3]:

ec_cobra_model_modified, ec_dingo_model_modified = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


Read LP format model from file /tmp/tmpvu1pxei6.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


In [4]:

samples_dingo = sample_dingo(ec_dingo_model, reaction_in_rows = True, ess=2000)

samples_dingo_modified_model = sample_dingo(ec_dingo_model_modified, reaction_in_rows = True, ess=2000)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 8, ratio of the maximum singilar value over the minimum singular value = 966.889
phase 2: number of correlated samples = 500, effective sample size = 5, ratio of the maximum singilar value over the minimum singular value = 212.805
phase 3: number of correlated samples = 500, effective sample size = 17, ratio of the maximum singilar value over the minimum singular value = 79.0583
phase 4: number of correlated samples = 500, effective sample size = 29, ratio of the maximum singilar value over the minimum singular value = 165.873
phase 5: number of correlated samples = 500, effective sample size = 163, ratio of the maximum singilar value over the minimum singular value = 3.26284
phase 6: number of correlated samples = 500, effective sample size = 191, ratio of the maximum singilar value over

[5]maximum marginal PSRF: 1.01245


phase 1: number of correlated samples = 500, effective sample size = 4, ratio of the maximum singilar value over the minimum singular value = 1217.36
phase 2: number of correlated samples = 500, effective sample size = 164, ratio of the maximum singilar value over the minimum singular value = 1
phase 3: number of correlated samples = 2400, effective sample size = 740
phase 4: number of correlated samples = 2400, effective sample size = 916
phase 5: number of correlated samples = 2400, effective sample size = 736
[5]total ess 2560: number of correlated samples = 8200




[5]maximum marginal PSRF: 1.00111


In [ ]:

from statsmodels.stats.multitest import multipletests
from scipy import stats
import numpy as np
        

def significantly_altered_reactions(conditions=[], selected_comparisons = [(0, 1)], cobra_model=None, p_value_cutoff=0.05, fold_change_cutoff=2):
    cobra_reactions_str = [str(reaction.id) for reaction in cobra_model.reactions]
        
    # if provided sampling dataset has reactions as cols ==> transpose
    for condition in conditions:
        if condition.shape[1] == len(cobra_reactions_str):
            condition = condition.T
    
    # Store p-values for each row for KS test
    p_values = {row: [] for row in range(len(cobra_reactions_str)) }
    # Store ks-values for each row for KS test
    ks_values = {row: [] for row in range(len(cobra_reactions_str)) }
    # Store fold-change-values for each row for KS test
    fold_change_values = {row: [] for row in range(len(cobra_reactions_str)) }

    for row in range(len(cobra_reactions_str)):
        for i, j in selected_comparisons:
            
            ks, p = stats.ks_2samp(conditions[i][row], conditions[j][row], alternative='two-sided')
            p_values[row].append(p)
            ks_values[row].append(ks)
            
            fold_change = np.abs( (np.mean(conditions[i][row]) - np.mean(conditions[j][row]) ) / np.mean(conditions[j][row])+1e-8 )
            fold_change_values[row].append(fold_change)

    p_values_copy = list(p_values.values())
    flat_p_values = np.array(p_values_copy).flatten()
    
    fold_change_values = np.array(list(fold_change_values.values()))


    # Apply FDR correction
    _, corrected_p_values, _, _ = multipletests(flat_p_values, method='fdr_bh')

    # Reshape corrected p-values back to the original matrix shape
    corrected_p_values = corrected_p_values.reshape(np.array(p_values_copy).shape)
    
    significant_diff_indices = np.where(
    np.logical_and(
        corrected_p_values < p_value_cutoff,
        fold_change_values > fold_change_cutoff
    ) )[0]
    
    significant_diff_reactions = [cobra_reactions_str[i] for i in significant_diff_indices]
    
    not_significant_diff_indices = np.where(corrected_p_values >= p_value_cutoff)[0]
    not_significant_diff_reactions = [cobra_reactions_str[i] for i in not_significant_diff_indices]
    
    return significant_diff_indices, significant_diff_reactions, not_significant_diff_reactions



In [ ]:


conditions=[samples_dingo, samples_dingo_modified_model]
selected_comparisons = [(0, 1)]
print(len(selected_comparisons[0]))

# I have to add FOLD CHANGE
# fva
_, significant_diff_reactions, not_significant_diff_reactions = significantly_altered_reactions(conditions, 
                                                                                                selected_comparisons, 
                                                                                                cobra_model=ec_cobra_model,
                                                                                                fold_change_cutoff=0.2)


print(significant_diff_reactions)
print(len(significant_diff_reactions))


2


/tmp/ipykernel_14901/306431553.py:28: RuntimeWarning: invalid value encountered in double_scalars
  fold_change = np.abs( (np.mean(conditions[i][row]) - np.mean(conditions[j][row])) / np.mean(conditions[j][row])+1e-8)


['PFK', 'PFL', 'PGI', 'PGL', 'ACALD', 'AKGt2r', 'PIt2r', 'ALCD2x', 'ACALDt', 'ACKr', 'PPC', 'ACONTa', 'ACONTb', 'ATPM', 'PPCK', 'ACt2r', 'PPS', 'ADK1', 'AKGDH', 'PTAr', 'PYK', 'BIOMASS_Ecoli_core_w_GAM', 'PYRt2', 'CO2t', 'RPE', 'CS', 'SUCCt2_2', 'CYTBD', 'D_LACt2', 'SUCCt3', 'ETOHt2r', 'SUCOAS', 'TALA', 'THD2', 'TKT1', 'TKT2', 'EX_ac_e', 'EX_acald_e', 'EX_akg_e', 'EX_co2_e', 'EX_etoh_e', 'EX_for_e', 'EX_glu__L_e', 'EX_h_e', 'EX_lac__D_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e', 'EX_pyr_e', 'EX_succ_e', 'FBP', 'FORt2', 'FORt', 'FUM', 'G6PDH2r', 'GLNS', 'GLUDy', 'GLUN', 'GLUSy', 'GLUt2r', 'GND', 'ICDHyr', 'ICL', 'LDH_D', 'MALS', 'MDH', 'ME1', 'ME2', 'NADH16', 'NADTRHD', 'NH4t', 'O2t', 'PDH']
73
